## References
* https://www.kaggle.com/kalashnimov/distilbert-baseline/notebook
* https://www.kaggle.com/sophiechampagne/jigsaw-baseline-models-v1/data

## Load Libraries

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import glob

# Params

In [ ]:
needed_cols = ['toxic', 'comment_text']

# Simple approach

In [ ]:
import string
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from gensim import utils
import gensim.parsing.preprocessing as gsp

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
# Loading train data

translated_train_files = glob.glob('/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-*-cleaned.csv')
translated_train_dfs = []
for filename in translated_train_files:
    df = pd.read_csv(filename, usecols=needed_cols)
    lang = re.findall('train-google-(.*)-cleaned.csv', filename)[0]
    df['lang'] = lang
    translated_train_dfs.append(df)

train_en = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv", usecols=needed_cols)
train_en['lang'] = 'en'

translated_train_dfs.append(train_en)
    
#train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
#train2.toxic = train2.toxic.round().astype(int)
#train2['lang'] = 'en'

train_df = pd.concat(translated_train_dfs).sample(n=400000).reset_index(drop=True)
# len 1563650
# train_df = pd.concat(translated_train_dfs).reset_index(drop=True)


del df, translated_train_dfs, train_en
gc.collect()

In [ ]:
train_df['toxic'].value_counts()

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): 
    return re_tok.sub(r' \1 ', s).split()
 
filters = [
           gsp.strip_tags, #remove tags 
           gsp.strip_punctuation, #remove punctuation
           gsp.strip_multiple_whitespaces, #standarized the spaces 
           gsp.strip_numeric,
           gsp.remove_stopwords, #stop words  
           gsp.strip_short, 
           gsp.stem_text #stemming 
          ]

def clean_text(s):
    s = str(s).lower() #lower case for all words
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

In [ ]:
#clean the text first 
train_df['comment_text'].fillna("unknown", inplace=True)
train_df["comment_text"] = train_df["comment_text"].apply(clean_text)

#split the train and test from the whole table 
# x_train, x_test, y_train, y_test = train_test_split(train_df["comment_text"], train_df['toxic'], test_size=0.3)

In [ ]:
#vectorization of the model 
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)

pipeline = Pipeline([
    ('tfidf', vec),
    ('logreg', LogisticRegression(penalty='elasticnet')),
])

parameters = {
    'tfidf__max_features': [5000], #[None, 1000, 5000, 50000], checked
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or unigrams + bigrams
    'logreg__penalty' : ['l1', 'l2'],
    'logreg__C' : np.logspace(1e-3, 1e3, 20),
    'logreg__solver' : ['liblinear'],
}

In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, 
                                   scoring=['roc_auc'],
                                   cv=cv, n_iter=10, refit='roc_auc')

In [ ]:
%%time

random_search.fit(train_df["comment_text"], train_df['toxic']);

del train_df; gc.collect()

In [ ]:
pd.DataFrame(random_search.cv_results_).sort_values('mean_test_roc_auc', ascending=False)

In [ ]:
val_df = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
val_df['comment_text'].fillna("unknown", inplace=True)
val_df["comment_text"] = val_df["comment_text"].apply(clean_text)

#score the validation dataset
y_valid = val_df['toxic']
y_pred_valid = random_search.best_estimator_.predict_proba(val_df["comment_text"])
# print('Testing accuracy %s' % accuracy_score(y_valid, y_pred_valid))
# print('Testing F1 score: {}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
print('Testing AUC score %s' % roc_auc_score(y_valid, y_pred_valid[:, 1]))

del val_df, y_valid, y_pred_valid; gc.collect()

In [ ]:
test_subm_df = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
test_subm_df["comment_text"] = test_subm_df["content"].apply(clean_text)

#score the submission file 
y_pred_submission = random_search.best_estimator_.predict_proba(test_subm_df["comment_text"])
#load the sample submission file 
sample_sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

submid = pd.DataFrame({'id': sample_sub["id"]})
submission = pd.concat([submid, pd.DataFrame(y_pred_submission[:, 1],columns=['toxic'])], axis=1)
display(submission.head())
submission['toxic'].hist(bins=100, log=False, alpha=1);
submission.to_csv('submission_logreg.csv', index=False)

del test_subm_df, y_pred_submission, sample_sub, submid, submission; gc.collect()

# Distil Bert

In [ ]:
!pip install -U transformers

In [ ]:
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from tqdm.notebook import tqdm
import tokenizers
from tokenizers import BertWordPieceTokenizer

## Helper functions

In [ ]:
def clean_text(text):
    text = str(text)
    text = re.sub(r'[0-9"]', '', text) # number
    text = re.sub(r'#[\S]+\b', '', text) # hash
    text = re.sub(r'@[\S]+\b', '', text) # mention
    text = re.sub(r'https?\S+', '', text) # link
    text = re.sub(r'\s+', ' ', text) # multiple white spaces
#     text = re.sub(r'\W+', ' ', text) # non-alphanumeric
    return text.strip()

def text_process(text):
    ws = text.split(' ')
    if(len(ws)>160):
        text = ' '.join(ws[:160]) + ' ' + ' '.join(ws[-32:])
    return text

def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    print('encoding with', tokenizer)
    
    # for transformers 3.5
    if isinstance(tokenizer, transformers.tokenization_distilbert.DistilBertTokenizer) or \
        isinstance(tokenizer, transformers.DistilBertTokenizerFast):
    #     tokenizer.enable_truncation(max_length=maxlen)
    #     tokenizer.enable_padding(max_length=maxlen)
        all_ids = []

        for i in tqdm(range(0, len(texts), chunk_size)):
            text_chunk = texts[i:i+chunk_size].tolist()
    #         encs = tokenizer.encode_batch(text_chunk)
            encs = tokenizer(text_chunk, padding='max_length', truncation=True, max_length=maxlen)
    #         all_ids.extend([enc.ids for enc in encs])
            all_ids.extend(encs['input_ids']) 
        
    elif isinstance(fast_tokenizer, tokenizers.implementations.bert_wordpiece.BertWordPieceTokenizer): 
        tokenizer.enable_truncation(max_length=maxlen)
        tokenizer.enable_padding(max_length=maxlen)
        all_ids = []

        for i in tqdm(range(0, len(texts), chunk_size)):
            text_chunk = texts[i:i+chunk_size].tolist()
            encs = tokenizer.encode_batch(text_chunk)
            all_ids.extend([enc.ids for enc in encs])

    
    return np.array(all_ids)

In [ ]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')

save_path = '/kaggle/working/distilbert_base_cased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

# "faster as the tokenizers from transformers because they are implemented in Rust."
# fast_tokenizer = BertWordPieceTokenizer('distilbert_base_cased/vocab.txt', lowercase=False)
fast_tokenizer = tokenizer

## TPU config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Configuration
AUTO = tf.data.experimental.AUTOTUNE
SHUFFLE = 2048
EPOCHS1 = 20
EPOCHS2 = 4
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
VERBOSE = 2

## Read data

may be switched from original to multilingual

In [ ]:
%%time 

# # Label distributions are pretty the same in different train sets, so let's make full use of both.  
# # If balanced subsample is needed, it may be done with the func below


# train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv", 
#                      usecols=needed_cols)
# train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv",
#                      usecols=needed_cols)
# # convert to binary
# train2.toxic = train2.toxic.round().astype(int)
# display(train1['toxic'].value_counts(normalize=True))
# display(train2['toxic'].value_counts(normalize=True))

# def downsample(df, negative_ratio=1):
#     """Subsample the train dataframe to 50%-50%"""
#     ds_df= pd.concat([
#         df.query('toxic==1'),
#         df.query('toxic==0').sample(sum(df['toxic'])*negative_ratio)
#     ])
    
#     return ds_df

# train_df = pd.concat([
#     train1[needed_cols],
#     train2[needed_cols]
# ])
# train_df['toxic'] = train_df['toxic']

# train_df = downsample(train_df, negative_ratio=3)

# display(train_df.head(5), train_df.shape)

# y_train = train_df['toxic'].values

# del train1, train2; gc.collect()




# Loading train data

translated_train_files = glob.glob('/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-*-cleaned.csv')
translated_train_dfs = []
for filename in translated_train_files:
    df = pd.read_csv(filename, usecols=needed_cols)
    lang = re.findall('train-google-(.*)-cleaned.csv', filename)[0]
    df['lang'] = lang
    translated_train_dfs.append(df)

train_en = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv", usecols=needed_cols)
train_en['lang'] = 'en'

translated_train_dfs.append(train_en)
    
#train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
#train2.toxic = train2.toxic.round().astype(int)
#train2['lang'] = 'en'

train_df = pd.concat(translated_train_dfs).sample(n=600000).reset_index(drop=True)
# train_df = pd.concat(translated_train_dfs).reset_index(drop=True)


y_train = train_df['toxic'].values


del df, translated_train_dfs, train_en
gc.collect()

In [ ]:
def len_sent(data):
    return len(data.split())
train_df["num_words_comment_text"] = train_df["comment_text"].apply(lambda x : len_sent(x))
sns.kdeplot(train_df[train_df["toxic"] == 0]["num_words_comment_text"].values, shade = True, color = "red", label='non_toxity')
sns.kdeplot(train_df[train_df["toxic"] == 1]["num_words_comment_text"].values, shade = True, color = "blue", label='toxity')

del train_df['toxic']; gc.collect()

In [ ]:
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: clean_text(x))
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: text_process(x))
x_train = fast_encode(train_df['comment_text'].astype(str), fast_tokenizer, maxlen=MAX_LEN)

## Build dataset objects

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(SHUFFLE)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

del x_train; gc.collect()


valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
valid['comment_text'] = valid.apply(lambda x: clean_text(x['comment_text']), axis=1)
valid['comment_text'] = valid['comment_text'].apply(lambda x: text_process(x))
x_valid = fast_encode(valid['comment_text'].astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_valid = valid['toxic'].values

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

del x_valid; gc.collect()

## Callbacks

In [ ]:
lrs = ReduceLROnPlateau(monitor='val_auc', mode ='max', factor = 0.7, min_lr= 1e-7, verbose = 1, patience = 2)
es1 = EarlyStopping(monitor='val_auc', mode='max', verbose = 1, patience = 5, restore_best_weights=True)
es2 = EarlyStopping(monitor='auc', mode='max', verbose = 1, patience = 1, restore_best_weights=True)
callbacks_list1 = [lrs,es1]
callbacks_list2 = [lrs,es2]

## Build model

In [ ]:
def build_model(transformer, max_len=512):

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.4)(cls_token)
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
    return model

## Load model into TPU

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

## Run model

In [ ]:
# not train on order to save memory
n_steps = len(y_train) // (BATCH_SIZE*8)

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS1,
    callbacks=callbacks_list1,
    verbose=VERBOSE
)

del train_dataset; gc.collect()

In [ ]:
train_history_df = pd.DataFrame.from_dict(train_history.history)
train_history_df

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(train_history_df['val_auc'], label='val')
plt.plot(train_history_df['auc'], label='train')
plt.legend(fontsize=15)

In [ ]:
n_steps = len(y_valid) // (BATCH_SIZE)

train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS2,
    callbacks=callbacks_list2,
    verbose=VERBOSE
)

In [ ]:
train_history2_df = pd.DataFrame.from_dict(train_history_2.history)
train_history2_df

# Submission

In [ ]:
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
test['content'] = test.apply(lambda x: clean_text(x['content']), axis=1)
test['content'] = test['content'].apply(lambda x: text_process(x))
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub['toxic'].hist(bins=100, log=False, alpha=1)
sub.to_csv('submission_distilbert.csv', index=False)

# Blending

In [ ]:
sub_logreg = pd.read_csv('submission_logreg.csv')
display(sub_logreg.head())

In [ ]:
sub_distilbert = pd.read_csv('submission_distilbert.csv')
display(sub_distilbert.head())

In [ ]:
p = 1

blend_sub = sub_logreg.copy()

blend_sub['toxic'] = \
    sub_logreg['toxic']**p * 0.15 + \
    sub_distilbert['toxic']**p * 0.85

In [ ]:
plt.figure(figsize=(15, 6))
plt.hist(sub_distilbert['toxic'], bins=100, log=False, alpha=0.3, label='distilbert');
plt.hist((sub_distilbert['toxic']**p), bins=100, log=False, alpha=0.3, label='distilbert power_'+str(p));
plt.hist(sub_logreg['toxic'], bins=100, log=False, alpha=0.3, label='logreg');
plt.hist((sub_logreg['toxic']**p), bins=100, log=False, alpha=0.3, label='logreg power_'+str(p));
plt.hist(blend_sub['toxic'], bins=100, log=False, alpha=0.3, label='blend');
plt.legend(fontsize=15);

In [ ]:
blend_sub.head()
blend_sub.to_csv('submission.csv', index=False)
# sub_distilbert.to_csv('submission.csv', index=False)